In [ ]:
from pyspark.context import SparkContext
sc = SparkContext()


In [ ]:
import re


In [ ]:
# Читаем файл
mv = sc.textFile('file:///home/kulkovil/test/mv.csv')

# ['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
# '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance',
# '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [ ]:
# Отделяем имена: разбиваем по запятым, берём все элементы кроме первого и последнего
# (в названии фильма могут быть запятые), объединяем обратно в строку по запятой
# и регулярным выражением удаляем год выпуска (если есть)
mv_names = mv.map(lambda x: re.sub(r'(\s\(\d{4}\))', '', ','.join(x.split(',')[1:-1])))

# ['title', 'Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale']


In [ ]:
# Разбиваем названия на слова по символам, отличным от букво-цифр,
# приводим к нижнему регистру и формируем ключи для подсчёта
mv_words = mv_names.flatMap(lambda x: re.split(r'[^\w\'$#]', x)).filter(lambda x: x).map(lambda x: (x.lower(), 1))

# [('title', 1), ('toy', 1), ('story', 1), ('jumanji', 1), ('grumpier', 1)]


In [ ]:
# Подсчитываем слова по ключам, сортируем по убыванию количества
mv_count = mv_words.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False)

# [('the', 3113), ('of', 975), ('a', 646), ('and', 355), ('in', 306)]
